In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from zipfile import ZipFile
import os, glob
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
os.chdir('/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes')
X = []
y = []
for i in tqdm(os.listdir()):
  img = cv2.imread(i)
    img = cv2.resize(img, (224, 224))
      X.append(img)
        y.append((i[0:1]))
          print(i[0:1])

          os.chdir('/kaggle/input/brain-mri-images-for-brain-tumor-detection/no')
          for i in tqdm(os.listdir()):
            img = cv2.imread(i)
              img = cv2.resize(img, (224, 224))
                X.append(img)
                  y.append((i[0:1]))
                    print(i[0:1])

                    for i in range(1,99):
                      y.append('N')
                      print(y)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
    plt.xticks([])
      plt.yticks([])
        plt.grid(False)
          plt.imshow(X[i], cmap=plt.cm.binary)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(4):
  plt.subplot(1, 4,i+1)
    plt.imshow(X[i], cmap= "gray")
      plt.axis('off')
        plt.show()

In [ ]:
print("X = ", len(X))
y = y[:253]
print("y = ", len(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                                                                        test_size= 0.3,
                                                                                                                                                            random_state= 42)
                                                                                                                                                            print("Shape of an image in X_train = ", X_train[0].shape)
                                                                                                                                                            print("Shape of an image in X_test = ", X_test[0].shape)

In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

In [ ]:
from keras.applications import vgg16

img_rows, img_cols = 224, 224

vgg = vgg16.VGG16(weights = 'imagenet',
                  include_top = False,
                                    input_shape = (img_rows, img_cols, 3))

                                    # Here we freeze the last 4 layers
                                    # Layers are set to trainable as True by default
                                    for layer in vgg.layers:
                                      layer.trainable = False

                                      # Printing out the layers
                                      for (i, layer) in enumerate(vgg.layers):
                                        print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
from keras.layers import GlobalAveragePooling2D

def lw(bottom_model,
       num_classes):
         top_model = bottom_model.output
           top_model = GlobalAveragePooling2D()(top_model)
             top_model = Dense(1024, activation = "relu")(top_model)
               top_model = Dense(1024, activation = "relu")(top_model)
                 top_model = Dense(512, activation = "relu")(top_model)
                   top_model = Dense(num_classes, activation = "softmax")(top_model)
                     return top_model

In [ ]:
from keras.models import Model


num_classes = 2
FC_Head = lw(vgg,
             num_classes)

             # This is the input shape of the data
             model = Model(inputs = vgg.input,
                           outputs = lw(vgg,
                                                      num_classes))
                                                      model.summary()

In [ ]:
num_classes = y_train.shape[1]
FC_Head = lw(vgg, num_classes)
model = Model(inputs=vgg.input, outputs=FC_Head)
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy",
              optimizer = "adam",
                            metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                                        validation_data=(X_test, y_test),
                                                            verbose=1,
                                                                                initial_epoch=0)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(loc = 0)
plt.grid(True)
plt.show()